In [191]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [192]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def print_values(self):
        print("\n")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

# Please fill in all inputs and their units below
# v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v 

# Constants
<br>For Reference: Units Set: psi, in2, in4, lbf

In [193]:
YOUNGSMODULUS = 30 * (10**6)
YM_UNIT = "psi"

MOI = 480
MOI_UNIT = "in2"

nArray = np.array([[1,2], [2,3]])
print(f"number of elements: {len(nArray)}")

coords = np.array([[0,0], [6,0], [12,0]])

LENGTHS_UNIT = "in"


# list of lengths corresponding to each element
#LENGTHS = [6, 6]

#LENGTHS = [(LENGTH * 12) for LENGTH in LENGTHS ] #convert to inches

#print(LENGTHS)

HINGE_NODES = [2]


number of elements: 2


# Boundary Conditions
<br>for v & phi: [node#, v, phi]
<br>for forces: [node#, Fy, moment]
<br><br>Please note, node# are inputted straight from the diagram - do not adjust for python counting

In [194]:
v_phi_bcs = np.array([[1, 0, 0],[3, 0, 0]])

forceMoment_BCs = np.array([[2, -1200, 0]])

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [195]:
E = Input("Young's Modulus", YOUNGSMODULUS, YM_UNIT)
E.print_values()

I_ = Input("Moment of Inertia", MOI, MOI_UNIT)
I_.print_values()



nodes = Input("Node numbers", nArray, "None")
nodes.print_values()

nodeCoordinates = Input("Node Coordinates (x, y)", coords, "None")
nodeCoordinates.print_values()

HINGE_NODES = [x - 1 for x in HINGE_NODES]
print(HINGE_NODES)



name: Young's Modulus
value: 30000000
unit: psi


name: Moment of Inertia
value: 480
unit: in2


name: Node numbers
value: [[1 2]
 [2 3]]
unit: None


name: Node Coordinates (x, y)
value: [[ 0  0]
 [ 6  0]
 [12  0]]
unit: None
[1]


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [196]:
count_elems = len(nodes.value)
print(f"number of elements: {count_elems}")

count_nodes = nodes.value.max()
print(f"number of nodes: {count_nodes}")

dimension = (2*count_nodes) + len(HINGE_NODES)
print(f"dimension = {dimension}")

number of elements: 2
number of nodes: 3
dimension = 7


In [197]:
LENGTHS = np.zeros(count_elems)
print(LENGTHS)

for i in range(count_elems):

    connection = nodes.value[i]
    N1 = connection[0]
    N2 = connection[1]

    x1 = (nodeCoordinates.value[N1-1])[0]
    y1 = (nodeCoordinates.value[N1-1])[1]

    x2 = (nodeCoordinates.value[N2-1])[0]
    y2 = (nodeCoordinates.value[N2-1])[1]

    LENGTHS[i] = get_element_length(x1, y1, x2, y2)

    print(f"element {i+1}: length = {LENGTHS[i]} (raw value)")

LENGTHS *= 12
print(LENGTHS)

lengths = Input("Lengths", LENGTHS, LENGTHS_UNIT)
lengths.print_values()


[0. 0.]
element 1: length = 6.0 (raw value)
element 2: length = 6.0 (raw value)
[72. 72.]


name: Lengths
value: [72. 72.]
unit: in


two reactions per node
<br>v_phi_list -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.


In [198]:
v_phi_list = [None]*(dimension-1)
print("displacement bcs: [node number, y disp (v), angle of rotation (phi)]")
print(v_phi_bcs)

adjust_array(v_phi_list, v_phi_bcs, "displacements")
for node in HINGE_NODES:
    v_phi_list.insert((node*2)+2, None)
print("v & phi list:")
print(v_phi_list)

displacement bcs: [node number, y disp (v), angle of rotation (phi)]
[[1 0 0]
 [3 0 0]]
adjusting array to account for: displacements boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
Case 2 for BC 2
done adjusting displacements array!
v & phi list:
[0, 0, None, None, None, 0, 0]


Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [199]:
forceList = [0]*(count_nodes*2)
#forceList = np.zeros(((count_nodes*2), 1))

nodesWithLoads = []

loadedNodes = 0

for i in range(len(forceMoment_BCs)):
    if forceMoment_BCs[i,1] != None or forceMoment_BCs[i,2] != None:
        loadedNodes += 1
        nodesWithLoads.append((forceMoment_BCs[i,0] - 1))

#add in extra reactions for hinges
for node in HINGE_NODES:
    forceList.insert((node*2)+2, 0)

for i in range(len(v_phi_list)):
    if v_phi_list[i] == 0:
        forceList[i] = None
    elif v_phi_list[i] is None:
        forceList[i] = 0

#adjust force array given concentrated loading conditions
adjust_array(forceList, forceMoment_BCs, "forces")

print(f"number of nodes with concentrated loads: {loadedNodes}")
print(f"at nodes: {nodesWithLoads}")

print(f"force list: \n{forceList}")

adjusting array to account for: forces boundary conditions
case 1 for BC 1
case 4 for BC 1
done adjusting forces array!
number of nodes with concentrated loads: 1
at nodes: [1]
force list: 
[None, None, -1200, 0, 0, None, None]


Global [k] matrix is a symmetric array of shape (count_nodes x 2, count_nodes x 2)
<br>beam constants are one per element, stored in a numElemes * 1 array.

In [200]:
global_K = np.zeros((dimension, dimension))
print(f"Global K array dimensions: {global_K.shape}")

beam_constant = np.zeros((count_elems,1))
for i in range(count_elems):
    print(f"E = {E.value}, I = {I_.value}, length for element {i+1} = {lengths.value[i]}")
    beam_constant[i] = (E.value*I_.value)/(lengths.value[i]**3)

print(f"beam constants = \n{beam_constant}")

Global K array dimensions: (7, 7)
E = 30000000, I = 480, length for element 1 = 72.0
E = 30000000, I = 480, length for element 2 = 72.0
beam constants = 
[[38580.24691358]
 [38580.24691358]]


## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [201]:
for i in range(count_elems):
    local_K = create_beam_local_k(lengths.value[i])
    local_K *= beam_constant[i]
    print(f"\nk matrix for element {i+1}: \n{local_K}")
    #print(local_K)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starting at node: {startNode} and ending at node: {endNode}")
    #print(np.vsplit(local_K, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, global_K, 0, 0, local_K)
    block_array_adjust(En, Sn, global_K, 2, 0, local_K)
    block_array_adjust(Sn, En, global_K, 0, 2, local_K)
    block_array_adjust(En, En, global_K, 2, 2, local_K)

print(f"\nGlobal K matrix: \n{global_K}")
unmodified_global_K = global_K.copy()
check_symmetric(global_K)


k matrix for element 1: 
[[ 4.62962963e+05  1.66666667e+07 -4.62962963e+05  1.66666667e+07]
 [ 1.66666667e+07  8.00000000e+08 -1.66666667e+07  4.00000000e+08]
 [-4.62962963e+05 -1.66666667e+07  4.62962963e+05 -1.66666667e+07]
 [ 1.66666667e+07  4.00000000e+08 -1.66666667e+07  8.00000000e+08]]
Element 1 starting at node: 1 and ending at node: 2

k matrix for element 2: 
[[ 4.62962963e+05  1.66666667e+07 -4.62962963e+05  1.66666667e+07]
 [ 1.66666667e+07  8.00000000e+08 -1.66666667e+07  4.00000000e+08]
 [-4.62962963e+05 -1.66666667e+07  4.62962963e+05 -1.66666667e+07]
 [ 1.66666667e+07  4.00000000e+08 -1.66666667e+07  8.00000000e+08]]
Element 2 starting at node: 2 and ending at node: 3

Global K matrix: 
[[ 4.62962963e+05  1.66666667e+07 -4.62962963e+05  1.66666667e+07
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.66666667e+07  8.00000000e+08 -1.66666667e+07  4.00000000e+08
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.62962963e+05 -1.66666667e+07  9.25925926e+05  0.

In [202]:
j = 0
while j < count_nodes*2:
    print(f"{j}")
    #if j in nodesWithLoads:
     #   j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #global_K to reflect the changes.
    if v_phi_list[j] == 0:
        print(f"u = 0 at row: {j}")
        forceList[j] = 0 
        global_K[j] = 0
        global_K[j,j] = 1
        j+=1
    elif v_phi_list[j] is None:
        print(f"none detected at row {j}")
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print(f"row {j} altered")
        print(v_phi_list[j])
        forceList[j] = v_phi_list[j]
        for c in range(count_nodes):
            if (global_K[j,c] > 0) == True:
                global_K[j,c] = 1
            else:
                global_K[j,c] = 0
        j+=1

                
print(global_K)
print(forceList)

0
u = 0 at row: 0
1
u = 0 at row: 1
2
none detected at row 2
3
none detected at row 3
4
none detected at row 4
5
u = 0 at row: 5
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.62962963e+05 -1.66666667e+07  9.25925926e+05  0.00000000e+00
  -4.62962963e+05  1.66666667e+07  0.00000000e+00]
 [ 1.66666667e+07  4.00000000e+08  0.00000000e+00  1.60000000e+09
  -1.66666667e+07  4.00000000e+08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.62962963e+05 -1.66666667e+07
   4.62962963e+05 -1.66666667e+07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[0, 0, -1200, 0, 0, 0, None]


In [203]:
#global_K.round(0)
global_K_inv = inv(global_K)
#print(global_K_inv)

node = 1

uFinal = np.matmul(global_K_inv, forceList)

for i in range(len(uFinal)):
    if check_even(i) == True:
        print(f"u{node} = {uFinal[i]}")
    else:
        print(f"v{node} = {uFinal[i]}\n")
        node += 1 

LinAlgError: Singular matrix

Forces can be found from [K]*[u]

In [ ]:
forcesFinal = np.matmul(unmodified_global_K, uFinal)
for i in range(len(forceList)):
    if forceList[i] not in [forcesFinal[i], 0, None]:
        forcesFinal[i] = forceList[i]
print(f"Final Forces: \n{forcesFinal}")

node = 1

for i in range(len(forcesFinal)):
    if check_even(i) == True:
        print(f"fx{node} = {forcesFinal[i]}")
    else:
        print(f"fy{node} = {forcesFinal[i]}")
        node += 1 


## Finding Stresses

In [ ]:
stressFinal = np.zeros((count_elems, 1))
print(stressFinal)
for i in range(count_elems):
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starts at node: {startNode} and ends at node: {endNode}")
    #print(np.vsplit(local_K, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    print(f"adjusted start node: {Sn} adjusted end node: {En}")
    trigMatrix = create_truss_stress_trig_matrix(angles.value[i])
    print(f"trigMatrix = \n{trigMatrix}")
    local_uMatrix = np.array([
        uFinal[Sn], 
        uFinal[(Sn + 1)], 
        uFinal[En], 
        uFinal[(En+1)]
        ])
    print(f"local u matrix: \n{local_uMatrix}")
    localLength = np.array([[(-1/lengths.value[i]), 1/lengths.value[i]]])
    print(f"local length matrix: \n{localLength}")

    indStress = (trigMatrix @ local_uMatrix)
    indStress = localLength @ indStress
    indStress *= E.value

    stressFinal[i] = indStress
    
    
    print(f"Stress in element = {indStress}")

    

In [ ]:
print(stressFinal)

In [ ]:
np.set_printoptions(suppress=True, formatter={'float':"{:0.4f}".format})
printTrussNodalDisplacements(uFinal, lengths.unit)
printStresses(stressFinal,E.unit)